# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
from datetime import timedelta
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Format today and yesterday into strings for filename creation
format = "%m-%d-%Y"
now = datetime.now()
today = now.strftime(format)
yesterday = (now - timedelta(days=1)).strftime(format)

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    jh_live_date = today
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    jh_live_date = yesterday
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
print(jh_live_date)

08-02-2022


In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2022-08-03 04:20:59,33.93911,67.709953,186120,7751,NaN,NaN,Afghanistan,478.109252,4.164518
1,NaN,NaN,NaN,Albania,2022-08-03 04:20:59,41.15330,20.168300,313582,3550,NaN,NaN,Albania,10896.587671,1.132080
2,NaN,NaN,NaN,Algeria,2022-08-03 04:20:59,28.03390,1.659600,267657,6876,NaN,NaN,Algeria,610.377728,2.568960
3,NaN,NaN,NaN,Andorra,2022-08-03 04:20:59,42.50630,1.521800,45508,153,NaN,NaN,Andorra,58898.595742,0.336205
4,NaN,NaN,NaN,Angola,2022-08-03 04:20:59,-11.20270,17.873900,102301,1912,NaN,NaN,Angola,311.264425,1.868994


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,7/24/22,7/25/22,7/26/22,7/27/22,7/28/22,7/29/22,7/30/22,7/31/22,8/1/22,8/2/22
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,184587,184819,185086,185272,185393,185481,185552,185749,185930,186120
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,304890,305123,306789,308050,309278,310362,311381,312097,312375,313582
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,266839,266916,267010,267096,267194,267287,267374,267454,267546,267657


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,7/24/22,7/25/22,7/26/22,7/27/22,7/28/22,7/29/22,7/30/22,7/31/22,8/1/22,8/2/22
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,7738,7738,7742,7743,7743,7743,7745,7748,7751,7751
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,3527,3529,3532,3535,3538,3541,3543,3545,3548,3550
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,6876,6876,6876,6876,6876,6876,6876,6876,6876,6876


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,7/24/22,7/25/22,7/26/22,7/27/22,7/28/22,7/29/22,7/30/22,7/31/22,8/1/22,8/2/22
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,7/24/22,7/25/22,7/26/22,7/27/22,7/28/22,7/29/22,7/30/22,7/31/22,8/1/22,8/2/22
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,17268,17349,17392,17425,17455,17468,17468,17468,17468,17468
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,61268,61519,61632,61756,61851,61967,61967,61967,61967,61967
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,6205,6244,6258,6276,6293,6308,6308,6308,6308,6308


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,7/24/22,7/25/22,7/26/22,7/27/22,7/28/22,7/29/22,7/30/22,7/31/22,8/1/22,8/2/22
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,220,220,220,220,220,220,220,220,220,220
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,686,687,687,687,687,687,687,687,687,687
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,100,100,100,100,100,100,100,100,100,100


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
675,1001.0,Autauga,Alabama,US,2022-08-03 04:20:59,32.539527,-86.644082,17468,220,NaN,NaN,"Autauga, Alabama, US",31265.997244,1.259446
712,1075.0,Lamar,Alabama,US,2022-08-03 04:20:59,33.779950,-88.096680,4057,65,NaN,NaN,"Lamar, Alabama, US",29387.902934,1.602169
713,1077.0,Lauderdale,Alabama,US,2022-08-03 04:20:59,34.901719,-87.656247,26166,402,NaN,NaN,"Lauderdale, Alabama, US",28217.709670,1.536345


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,91585521,1030962,0.0
India,44067144,526477,0.0
France,34133344,153320,0.0


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,91585521,1030962,0.0,90554559.0,2022-08-03 04:20:59,37.934269,-91.444083
India,44067144,526477,0.0,43540667.0,2022-08-03 04:20:59,23.088275,81.806127
France,34133344,153320,0.0,33980024.0,2022-08-03 04:20:59,6.430808,-34.730285
Brazil,33890428,679010,0.0,33211418.0,2022-08-03 04:20:59,-12.669522,-48.480493
Germany,31044554,144360,0.0,30900194.0,2022-08-03 04:20:59,51.571844,10.277881


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,10761698,93609,0.0
Texas,7577212,89366,0.0
Florida,6833033,77734,0.0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,10761698,93609,0.0,10668089.0,2022-08-03 04:20:59,37.843962,-120.728594
Texas,7577212,89366,0.0,7487846.0,2022-08-03 04:20:59,31.660643,-98.653069
Florida,6833033,77734,0.0,6755299.0,2022-08-03 04:20:59,28.940755,-82.700744
New York,5838379,70381,0.0,5767998.0,2022-08-03 04:20:59,42.544151,-75.474183
Illinois,3578991,38889,0.0,3540102.0,2022-08-03 04:20:59,39.843618,-89.178330


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,3309285,32763,0.0
Florida,Miami-Dade,1401698,11102,0.0
Arizona,Maricopa,1383592,17616,0.0
Illinois,Cook,1341966,14604,0.0
Texas,Harris,1160026,11127,0.0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,3309285,32763,0.0,3276522.0,2022-08-03 04:20:59,34.308284,-118.228241,6037.0
Florida,Miami-Dade,1401698,11102,0.0,1390596.0,2022-08-03 04:20:59,25.611236,-80.551706,12086.0
Arizona,Maricopa,1383592,17616,0.0,1365976.0,2022-08-03 04:20:59,33.348359,-112.491815,4013.0
Illinois,Cook,1341966,14604,0.0,1327362.0,2022-08-03 04:20:59,41.841448,-87.816588,17031.0
Texas,Harris,1160026,11127,0.0,1148899.0,2022-08-03 04:20:59,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-07-31,185749,312097,267454,45508,102301,11,8773,9560307,425365,9435839,...,969803,243093,11724,535836,10779632,682096,535,11877,329483,256378
2022-08-01,185930,312375,267546,45508,102301,11,8773,9560307,425365,9473282,...,971728,243131,11724,536086,10781009,682096,535,11877,329483,256382
2022-08-02,186120,313582,267657,45508,102301,11,8773,9560307,426799,9517224,...,971728,243162,11730,536086,10783026,691065,535,11877,330575,256403


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-07-31,7748,3545,6876,153,1912,0,144,129369,8633,11865,...,7403,1637,14,5763,43093,5678,0,2151,4015,5577
2022-08-01,7751,3548,6876,153,1912,0,144,129369,8633,11957,...,7415,1637,14,5763,43093,5678,0,2151,4015,5577
2022-08-02,7751,3550,6876,153,1912,0,144,129369,8635,12026,...,7415,1637,14,5763,43094,5688,0,2151,4015,5578


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-07-31,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-08-01,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-08-02,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,7/24/22,7/25/22,7/26/22,7/27/22,7/28/22,7/29/22,7/30/22,7/31/22,8/1/22,8/2/22
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,17268,17349,17392,17425,17455,17468,17468,17468,17468,17468
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,4000,4013,4028,4036,4049,4057,4057,4057,4057,4057
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,25845,25949,26016,26068,26114,26166,26166,26166,26166,26166
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,8107,8145,8161,8178,8193,8208,8208,8208,8208,8208
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,40893,41072,41198,41300,41392,41460,41460,41460,41460,41460


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-07-31,1424411,283643,7471,2196429,903931,10702564,1596242,854582,294041,49,...,2198946,7533973,1010292,137649,21775,1954716,1729949,555107,1777707,170031
2022-08-01,1424411,283643,7471,2196429,904513,10715844,1599497,854934,294908,49,...,2202346,7562672,1010292,137649,21819,1962140,1740938,557371,1782325,170031
2022-08-02,1424411,283643,7766,2196429,905824,10761698,1601053,856003,295080,49,...,2202346,7577212,1010292,137649,21869,1965090,1740938,558300,1784213,171322


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                      \
Admin2         Autauga Baldwin Barbour  Bibb Blount Bullock Butler Calhoun   
2022-07-31       17468   61967    6308  6947  15998    2623   5547   34553   
2022-08-01       17468   61967    6308  6947  15998    2623   5547   34553   
2022-08-02       17468   61967    6308  6947  15998    2623   5547   34553   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2022-07-31         9388     5528  ...    7231   2085     8881     2082   
2022-08-01         9388     5528  ...    7231   2085     8881     2082   
2022-08-02         9388     5528  ...    7266   2092     8952     2110   

Province_State                                                     
Admin2         Sweetwater  Teton Uinta Unassigned Washakie Weston  
2022-07-31          11802  11229  5968          0     2565   1739  
2022-08-01          11802  11229  5968          0     2565   1739  
2022-08-02          11854  11304  5991          0     2574   1747  

[3 rows x 3336 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-07-31,19891,1309,33,30768,11719,93491,12986,11102,3024,0,...,27006,89260,4900,693,120,20918,13604,7156,14922,1856
2022-08-01,19891,1309,33,30768,11719,93530,12993,11102,3024,0,...,27030,89337,4900,693,120,20938,13652,7161,14936,1856
2022-08-02,19891,1309,33,30768,11728,93609,13006,11129,3026,0,...,27030,89366,4900,693,121,20969,13652,7163,14942,1856


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2022-07-31         220     687     100  105    247      54    129     638   
2022-08-01         220     687     100  105    247      54    129     638   
2022-08-02         220     687     100  105    247      54    129     638   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2022-07-31          165       87  ...     146     42       81       27   
2022-08-01          165       87  ...     146     42       81       27   
2022-08-02          165       87  ...     146     42       81       27   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2022-07-31            128    16    39          0       46     20  
2022-08-01            128    16    39          0       46     20  
2022-08-02            128    16    39          0       46     20  

[3 rows x 3336 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change().round(7)
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-07-31,0.001062,0.002299,0.000299,0.0,0.0,0.0,0.0,0.005548,0.000000,0.002924,...,0.000000,0.000173,0.000000,0.001044,0.000137,0.000000,0.0,0.0,0.000000,0.000008
2022-08-01,0.000974,0.000891,0.000344,0.0,0.0,0.0,0.0,0.000000,0.000000,0.003968,...,0.001985,0.000156,0.000000,0.000467,0.000128,0.000000,0.0,0.0,0.000000,0.000016
2022-08-02,0.001022,0.003864,0.000415,0.0,0.0,0.0,0.0,0.000000,0.003371,0.004639,...,0.000000,0.000128,0.000512,0.000000,0.000187,0.013149,0.0,0.0,0.003314,0.000082


In [60]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [61]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change().round(7)
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-07-31,0.000387,0.000565,0.0,0.0,0.0,NaN,0.0,0.000704,0.000000,0.001604,...,0.000000,0.0,0.0,0.000174,0.000000,0.000000,NaN,0.0,0.0,0.000000
2022-08-01,0.000387,0.000846,0.0,0.0,0.0,NaN,0.0,0.000000,0.000000,0.007754,...,0.001621,0.0,0.0,0.000000,0.000000,0.000000,NaN,0.0,0.0,0.000000
2022-08-02,0.000000,0.000564,0.0,0.0,0.0,NaN,0.0,0.000000,0.000232,0.005771,...,0.000000,0.0,0.0,0.000000,0.000023,0.001761,NaN,0.0,0.0,0.000179


In [62]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [63]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change().round(7)
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-07-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-08-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-08-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [65]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change().round(7)
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-07-31,0.0,0.0,0.000000,0.0,0.000416,0.000000,0.000000,0.000553,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2022-08-01,0.0,0.0,0.000000,0.0,0.000644,0.001241,0.002039,0.000412,0.002949,0.0,...,0.001546,0.003809,0.0,0.0,0.002021,0.003798,0.006352,0.004078,0.002598,0.000000
2022-08-02,0.0,0.0,0.039486,0.0,0.001449,0.004279,0.000973,0.001250,0.000583,0.0,...,0.000000,0.001923,0.0,0.0,0.002292,0.001504,0.000000,0.001667,0.001059,0.007593


In [66]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [67]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change().round(7)
jh_hist_uscounties_cases_pct.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2022-07-31         0.0     0.0     0.0  0.0    0.0     0.0    0.0     0.0   
2022-08-01         0.0     0.0     0.0  0.0    0.0     0.0    0.0     0.0   
2022-08-02         0.0     0.0     0.0  0.0    0.0     0.0    0.0     0.0   

Province_State                    ...  Wyoming                                \
Admin2         Chambers Cherokee  ...     Park    Platte  Sheridan  Sublette   
2022-07-31          0.0      0.0  ...  0.00000  0.000000  0.000000  0.000000   
2022-08-01          0.0      0.0  ...  0.00000  0.000000  0.000000  0.000000   
2022-08-02          0.0      0.0  ...  0.00484  0.003357  0.007995  0.013449   

Province_State                                                              
Admin2         Sweetwater     Teton     Uinta Unassigned  Washakie  Weston  
2022-07-31       0.000000  0.000000  0.000000        NaN  0.000000  0.0000  
2022-08-01       0.000000  0.000000  0.000000        NaN  0.000000  0.0000  
2022-08-02       0.004406  0.006679  0.003854        NaN  0.003509  0.0046  

[3 rows x 3336 columns]

In [68]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [69]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change().round(7)
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-07-31,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
2022-08-01,0.0,0.0,0.0,0.0,0.000000,0.000417,0.000539,0.000000,0.000000,NaN,...,0.000889,0.000863,0.0,0.0,0.000000,0.000956,0.003528,0.000699,0.000938,0.0
2022-08-02,0.0,0.0,0.0,0.0,0.000768,0.000845,0.001001,0.002432,0.000661,NaN,...,0.000000,0.000325,0.0,0.0,0.008333,0.001481,0.000000,0.000279,0.000402,0.0


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change().round(7)
jh_hist_uscounties_deaths_pct.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2022-07-31         0.0     0.0     0.0  0.0    0.0     0.0    0.0     0.0   
2022-08-01         0.0     0.0     0.0  0.0    0.0     0.0    0.0     0.0   
2022-08-02         0.0     0.0     0.0  0.0    0.0     0.0    0.0     0.0   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2022-07-31          0.0      0.0  ...     0.0    0.0      0.0      0.0   
2022-08-01          0.0      0.0  ...     0.0    0.0      0.0      0.0   
2022-08-02          0.0      0.0  ...     0.0    0.0      0.0      0.0   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2022-07-31            0.0   0.0   0.0        NaN      0.0    0.0  
2022-08-01            0.0   0.0   0.0        NaN      0.0    0.0  
2022-08-02            0.0   0.0   0.0        NaN      0.0    0.0  

[3 rows x 3336 columns]

In [72]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [73]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean().round(7)
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-07-31,0.000796,0.002906,0.000317,0.000254,1.100000e-06,0.0,0.000927,0.002791,0.000018,0.003598,...,0.000032,0.000200,-0.000027,0.000951,0.000149,0.000235,0.0,0.000591,0.000022,0.000034
2022-08-01,0.000885,0.001899,0.000331,0.000127,5.000000e-07,0.0,0.000463,0.001396,0.000009,0.003783,...,0.001009,0.000178,-0.000014,0.000709,0.000139,0.000117,0.0,0.000295,0.000011,0.000025
2022-08-02,0.000954,0.002881,0.000373,0.000064,3.000000e-07,0.0,0.000232,0.000698,0.001690,0.004211,...,0.000504,0.000153,0.000249,0.000354,0.000163,0.006633,0.0,0.000148,0.001663,0.000053


In [74]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [75]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean().round(7)
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-07-31,0.000271,0.000631,6.000000e-07,-0.0,1.000000e-07,NaN,1.000000e-07,0.000354,0.000004,0.004562,...,0.000034,0.0,0.0,0.000197,0.000003,0.000039,NaN,0.000233,0.000004,0.000051
2022-08-01,0.000329,0.000739,3.000000e-07,-0.0,1.000000e-07,NaN,1.000000e-07,0.000177,0.000002,0.006158,...,0.000828,0.0,0.0,0.000099,0.000002,0.000019,NaN,0.000116,0.000002,0.000026
2022-08-02,0.000164,0.000651,1.000000e-07,-0.0,0.000000e+00,NaN,0.000000e+00,0.000089,0.000117,0.005964,...,0.000414,0.0,0.0,0.000049,0.000012,0.000890,NaN,0.000058,0.000001,0.000102


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [77]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean().round(7)
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-07-31,-0.5,-0.499907,-0.496523,-0.498569,-0.493288,NaN,-0.499171,-0.498809,-0.499661,-0.499402,...,-0.499758,-0.496783,-0.5,-0.498237,-0.453466,-0.499858,NaN,-0.497938,-0.498727,-0.489078
2022-08-01,-0.5,-0.499907,-0.496523,-0.498569,-0.493288,NaN,-0.499171,-0.498809,-0.499661,-0.499402,...,-0.499758,-0.496783,-0.5,-0.498237,-0.453466,-0.499858,NaN,-0.497938,-0.498727,-0.489078
2022-08-02,-0.5,-0.499907,-0.496523,-0.498569,-0.493288,NaN,-0.499171,-0.498809,-0.499661,-0.499402,...,-0.499758,-0.496783,-0.5,-0.498237,-0.453466,-0.499858,NaN,-0.497938,-0.498727,-0.489078


In [78]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [79]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean().round(7)
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-07-31,0.00048,0.000274,0.000012,0.000249,0.000932,0.000639,0.000276,0.000793,0.000363,0.0,...,0.000356,0.000432,0.000368,0.000089,0.000504,0.000466,0.000104,0.000460,0.000325,0.000126
2022-08-01,0.00024,0.000137,0.000006,0.000125,0.000788,0.000940,0.001158,0.000602,0.001656,0.0,...,0.000951,0.002120,0.000184,0.000044,0.001262,0.002132,0.003228,0.002269,0.001461,0.000063
2022-08-02,0.00012,0.000068,0.019746,0.000062,0.001119,0.002609,0.001065,0.000926,0.001120,0.0,...,0.000476,0.002022,0.000092,0.000022,0.001777,0.001818,0.001614,0.001968,0.001260,0.003828


In [80]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [81]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean().round(7)
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2022-07-31      0.000339  0.000458  0.000647  0.000345  0.000350  0.000511   
2022-08-01      0.000170  0.000229  0.000323  0.000172  0.000175  0.000255   
2022-08-02      0.000085  0.000115  0.000162  0.000086  0.000088  0.000128   

Province_State                                         ...   Wyoming  \
Admin2            Butler   Calhoun Chambers  Cherokee  ...      Park   
2022-07-31      0.000355  0.000423  0.00056  0.000463  ...  0.000048   
2022-08-01      0.000177  0.000211  0.00028  0.000231  ...  0.000024   
2022-08-02      0.000089  0.000106  0.00014  0.000116  ...  0.002432   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2022-07-31      0.000107  0.000121  0.000145   0.000074  0.000115  0.000111   
2022-08-01      0.000053  0.000061  0.000073   0.000037  0.000057  0.000056   
2022-08-02      0.001705  0.004028  0.006761   0.002221  0.003368  0.001955   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2022-07-31      -0.503937  0.000124  0.000119  
2022-08-01      -0.503937  0.000062  0.000060  
2022-08-02      -0.503937  0.001785  0.002330  

[3 rows x 3336 columns]

In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [83]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean().round(7)
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-07-31,0.000016,0.0,0.000008,0.000072,0.000434,0.000200,0.000246,0.000020,0.000019,NaN,...,0.000163,0.000111,0.000206,0.000182,0.000000,0.000150,0.000119,0.000314,0.000086,0.000043
2022-08-01,0.000008,0.0,0.000004,0.000036,0.000217,0.000309,0.000393,0.000010,0.000009,NaN,...,0.000526,0.000487,0.000103,0.000091,0.000000,0.000553,0.001824,0.000506,0.000512,0.000021
2022-08-02,0.000004,0.0,0.000002,0.000018,0.000492,0.000577,0.000697,0.001221,0.000335,NaN,...,0.000263,0.000406,0.000052,0.000045,0.004167,0.001017,0.000912,0.000393,0.000457,0.000011


In [84]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [85]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean().round(7)
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State       Alabama                                               \
Admin2               Autauga   Baldwin Barbour Bibb Blount Bullock Butler   
2022-07-31      1.700000e-06  0.000014     0.0  0.0    0.0     0.0    0.0   
2022-08-01      8.000000e-07  0.000007     0.0  0.0    0.0     0.0    0.0   
2022-08-02      4.000000e-07  0.000003     0.0  0.0    0.0     0.0    0.0   

Province_State                            ... Wyoming                          \
Admin2         Calhoun Chambers Cherokee  ...    Park        Platte  Sheridan   
2022-07-31         0.0      0.0      0.0  ...     0.0  3.000000e-06 -0.000380   
2022-08-01         0.0      0.0      0.0  ...     0.0  1.500000e-06 -0.000190   
2022-08-02         0.0      0.0      0.0  ...     0.0  7.000000e-07 -0.000095   

Province_State                                                       \
Admin2         Sublette Sweetwater Teton Uinta Unassigned  Washakie   
2022-07-31          0.0   0.000124   0.0   0.0       -1.0  0.000710   
2022-08-01          0.0   0.000062   0.0   0.0       -1.0  0.000355   
2022-08-02          0.0   0.000031   0.0   0.0       -1.0  0.000178   

Province_State                
Admin2                Weston  
2022-07-31      1.000000e-07  
2022-08-01      0.000000e+00  
2022-08-02      0.000000e+00  

[3 rows x 3336 columns]

In [86]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [87]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

HTTPError: HTTP Error 403: Forbidden

In [ ]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [ ]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

In [ ]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [ ]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

In [ ]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

#### 